In [1]:
import boto3
import configparser
import time
import glob
import os
import pandas as pd
import json
import pyarrow.parquet as pq

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
KEY = config['AWS']['KEY']
SECRET = config['AWS']['SECRET']
REGION_NAME = config['AWS']['REGION_NAME']

DWH_CLUSTER_TYPE = config['DWH']['DWH_CLUSTER_TYPE']
DWH_NUM_NODES = config['DWH']['DWH_NUM_NODES']
DWH_NODE_TYPE = config['DWH']['DWH_NODE_TYPE']
DWH_IAM_ROLE_NAME = config['DWH']['DWH_IAM_ROLE_NAME']
DWH_CLUSTER_IDENTIFIER = config['DWH']['DWH_CLUSTER_IDENTIFIER']
DWH_DB = config['DWH']['DWH_DB']
DWH_DB_USER = config['DWH']['DWH_DB_USER']
DWH_DB_PASSWORD = config['DWH']['DWH_DB_PASSWORD']
DWH_PORT = config['DWH']['DWH_PORT']

YELLOW_TRIP_DATA = config['PATH']['YELLOW_TRIP_DATA']
GREEN_TRIP_DATA = config['PATH']['GREEN_TRIP_DATA']
FHVHV_TRIP_DATA = config['PATH']['FHVHV_TRIP_DATA']
FHVHV_TRIP_DATA = config['PATH']['FHVHV_TRIP_DATA']
TAXI_ZONE = config['PATH']['TAXI_ZONE']

pd.DataFrame({"Param":
              ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
            [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
})

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,t2.micro
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,test_iam_role


In [ ]:
s3  = boto3.client('s3',aws_access_key_id = KEY, aws_secret_access_key = SECRET, region_name = REGION_NAME )
response = s3.list_buckets()
print(response['Buckets'][0]['Name'])

ny-taxi-bucket-s3-1744458153689776500


In [39]:
tt = time.time_ns()
prefix = f'ny-taxi-bucket-s3-{tt}'
bucket_names = {prefix: 'unprocessed_reports, rides, locations'}
for k in bucket_names:
  print(prefix)
  print(k)

ny-taxi-bucket-s3-1744493321867458900
ny-taxi-bucket-s3-1744493321867458900


In [40]:
def create_s3_bucket(s3, b, folders):
  s3.create_bucket(Bucket = b, CreateBucketConfiguration={'LocationConstraint': REGION_NAME})
  if folders !=  ' ':
    fls = folders.split(',')
    for f in fls:
      s3.put_object(Bucket = b, Body = ' ', Key = f + '/')

tt = time.time_ns()
bucket_names = f'ny-taxi-bucket-s3-{tt}'
objects = 'unprocessed_reports, rides, locations'
create_s3_bucket(s3, bucket_names, objects)
print('S3 buckets created')

S3 buckets created


In [41]:
def upload_files_to_s3(s3, file, b, objects):
  key = objects + '/' + os.path.basename(file)
  response = s3.upload_file(file, b, key, ExtraArgs=None)
  return response

print('Uploading the local files to ny-taxi-bucket-s3 AWS S3 bucket...')
file = 'test_data.csv'
print('upload_file:', file)
upload_files_to_s3(s3, file, bucket_names, 'unprocessed_reports')
print(f'Files uploaded to {bucket_names} AWS S3 bucket')
print(f'ID: {tt}')

Uploading the local files to ny-taxi-bucket-s3 AWS S3 bucket...
upload_file: test_data.csv
Files uploaded to ny-taxi-bucket-s3-1744493321881726400 AWS S3 bucket
ID: 1744493321881726400
